# Factory Overrides

In [1]:
from pyuvm import *

class Producer(uvm_component):
    def build_phase(self):
        self.ap = uvm_analysis_port("ap", self)
        
    def run_phase(self):
        self.raise_objection()
        for nn in range(5):
            self.ap.write(nn)
        self.drop_objection()
        
class Consumer(uvm_subscriber):
    def write(self, datum):
        print(datum, end=" ")

class VertConsumer(Consumer):
    def write(self, datum):
        print(datum)

class IncConsumer(VertConsumer):
    def write(self, datum):
        print(datum+1)
        
class Env(uvm_env):
    def build_phase(self):
        self.pro  = Producer.create("pro", self)
        self.con   = Consumer.create("con", self)
        self.con2  = Consumer.create("con2", self)

    def connect_phase(self):
        self.pro.ap.connect(self.con)
        self.pro.ap.connect(self.con2)

class BaseTest(uvm_test):
    def build_phase(self):
        self.env = Env.create("env", self)

class VertTest(BaseTest):
    def build_phase(self):
        uvm_factory().set_type_override_by_type(Consumer, VertConsumer)
        super().build_phase()

class IncTest(BaseTest):
    def build_phase(self):
        uvm_factory().set_inst_override_by_type(Consumer, IncConsumer, self.get_full_name()+".env.con2")
        super().build_phase()
        
uvm_root().run_test("IncTest")
            

0 1
1 2
2 3
3 4
4 5


In [1]:
from pyuvm import *
from tb_pkg import *
from alu import *
import random

class FaultyScoreboard(Scoreboard):
    def run_phase(self):
        while True:
            rtxn = self.rslt_get_port.get()
            cmd = self.cmd_get_port.get()
            actual_result = rtxn.result
            predicted_result = TinyAluTlm.alu_op(cmd.A, cmd.B, cmd.op)
            # This part makes it faulty
            if random.randint(0,2) == 1:
                predicted_result += 1
            #
            if predicted_result == actual_result:
                self.logger.info(f"PASSED: 0x{cmd.A:02x} {cmd.op.name} 0x{cmd.B:02x} ="
                                 f" 0x{actual_result:04x}")
            else:
                self.logger.error(f"FAILED: 0x{cmd.A:02x} {cmd.op.name} 0x{cmd.B:02x} "
                                  f"= 0x{actual_result:04x} expected 0x{predicted_result:04x}")

class FaultyTest(AluTest):
    def build_phase(self):
        uvm_factory().set_type_override_by_type(Scoreboard, FaultyScoreboard)
        super().build_phase()

uvm_root().run_test("FaultyTest")

INFO: <ipython-input-1-20efdf654a5c>(16)[uvm_test_top.env.passive_agent.scoreboard]: PASSED: 0x91 XOR 0xbf = 0x002e
ERROR: <ipython-input-1-20efdf654a5c>(19)[uvm_test_top.env.passive_agent.scoreboard]: FAILED: 0x85 ADD 0x82 = 0x0107 expected 0x0108
INFO: <ipython-input-1-20efdf654a5c>(16)[uvm_test_top.env.passive_agent.scoreboard]: PASSED: 0x5f ADD 0xb6 = 0x0115
ERROR: <ipython-input-1-20efdf654a5c>(19)[uvm_test_top.env.passive_agent.scoreboard]: FAILED: 0x01 ADD 0x73 = 0x0074 expected 0x0075
ERROR: <ipython-input-1-20efdf654a5c>(19)[uvm_test_top.env.passive_agent.scoreboard]: FAILED: 0x14 MUL 0x6f = 0x08ac expected 0x08ad
ERROR: alu.py(32)[uvm_test_top.env.passive_agent.coverage]: Functional coverage error. Missed: {<Ops.NOP: 0>, <Ops.AND: 2>}


In [6]:
random.randint(0,1)

0